In [1]:
import pandas as pd
import numpy as np
import re
import re, string
from string import digits
import contractions

import warnings
warnings.filterwarnings('ignore')

import fasttext

In [2]:
d1 = pd.read_csv('data/B09GFLFMPS.csv')
d2 = pd.read_csv('data/B09GFLXVH9.csv')

data_raw = pd.concat([d1,d2],axis=0,ignore_index=True)
data = data_raw.copy()

In [3]:
data = data.drop_duplicates().reset_index(drop=True)

In [4]:
lng_model = fasttext.load_model('data/lid.176.ftz')
def langdet(st):
    if len(st)>1:
        return detect(st)
def fstlangdet(st):
    if len(st)>1:
        return lng_model.predict(st)[0][0].split('__')[2]
    

In [5]:
emoji_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags = re.UNICODE)

In [6]:
def titleBody(title,body,emoji_pattern):
    title =str(title).lower()
    body =str(body).lower()
    if (title !=body)|(title not in body):
        x = emoji_pattern.sub(r'', (title+' '+body).strip())
        return contractions.fix(x).replace('nan','')
    else:
        x = emoji_pattern.sub(r'', body.strip())
        return contractions.fix(x).replace('nan','')
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

In [7]:
data['review'] = data.apply(lambda x: titleBody(x['review_title'],x['review_body'],emoji_pattern),axis=1)
data['review'] = data['review'].apply(lambda x: re.compile('[%s]' % re.escape(string.punctuation)).sub(' ',(x)))
data['review'] = data['review'].apply(lambda x: x.translate(str.maketrans('', '', digits)))
data['review'] = data['review'].apply(lambda x: re.sub("\s\s+" , " ", x))
data['review'] = data['review'].str.strip()

In [8]:
data['fstlanguage'] = data['review'].apply(lambda x: fstlangdet(x))
#data_review = data[data['fstlanguage']=='en'][['review']]
data['fstlanguage'].value_counts()
data   = data[data['fstlanguage']=='en']

### Sentiment Analysis

In [147]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
nltk.download('sentiwordnet')
from nltk.corpus import sentiwordnet as swn

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


for sentiment analysis, i am not removing the stopwords 

In [148]:
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew



In [149]:
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}

data_review = data[['review']]
data_review['review_clean'] = data_review['review'].apply(clean)
data_review['POS tagged'] = data_review['review_clean'].apply(token_stop_pos)

In [150]:
wordnet_lemmatizer = WordNetLemmatizer()
data_review['Lemma'] = data_review['POS tagged'].apply(lemmatize)

In [151]:
data_review

,review,review_clean,POS tagged,Lemma
0,value for money if you are looking for a budge...,value for money if you are looking for a budge...,"[(value, n), (for, None), (money, n), (if, Non...",value for money if you be look for a budget ...
1,works a like a charm in this price range i get...,works a like a charm in this price range i get...,"[(works, v), (a, None), (like, None), (a, None...",work a like a charm in this price range i ge...
2,value for money i bought this for my mother wh...,value for money i bought this for my mother wh...,"[(value, n), (for, None), (money, n), (i, n), ...",value for money i buy this for my mother who...
3,good budget mobile good enough mobile in this ...,good budget mobile good enough mobile in this ...,"[(good, a), (budget, n), (mobile, n), (good, a...",good budget mobile good enough mobile in thi...
4,camera only camera quality not good,camera only camera quality not good,"[(camera, n), (only, r), (camera, n), (quality...",camera only camera quality not good
...,...,...,...,...
19946,good phone superb quality mobile phone and cam...,good phone superb quality mobile phone and cam...,"[(good, a), (phone, n), (superb, n), (quality,...",good phone superb quality mobile phone and c...
19947,exslent mobile sarvis is good,exslent mobile sarvis is good,"[(exslent, n), (mobile, n), (sarvis, n), (is, ...",exslent mobile sarvis be good
19948,best product in this price range good battery ...,best product in this price range good battery ...,"[(best, a), (product, n), (in, None), (this, N...",best product in this price range good batter...
19949,very fast delivery awesome,very fast delivery awesome,"[(very, r), (fast, r), (delivery, n), (awesome...",very fast delivery awesome


### Sentiment Analysis using TextBlob:
TextBlob is a Python library for processing textual data. It provides a consistent API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, and more.

The two measures that are used to analyze the sentiment are:

    Polarity – talks about how positive or negative the opinion is
    Subjectivity – talks about how subjective the opinion is, Subjective sentences generally refer to personal opinion, emotion, or judgment. 
    
TextBlob(text).sentiment gives us the Polarity, Subjectivity values.

    Polarity ranges from -1 to 1 (1 is more positive, 0 is neutral, -1 is more negative)
    Subjectivity ranges from 0 to 1(0 being very objective and 1 being very subjective)
    
    
Textblob will ignore the words that it doesn’t know, it will consider words and phrases that it can assign polarity to and averages to get the final score.

In [152]:
# function to calculate subjectivity
def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity
    # function to calculate polarity
def getPolarity(review):
    return TextBlob(review).sentiment.polarity

# function to analyze the reviews
def mapScore(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [153]:
data_review['Subjectivity'] = data_review['Lemma'].apply(getSubjectivity) 
data_review['Polarity'] = data_review['Lemma'].apply(getPolarity) 
data_review['TextBlobAnalysis'] = data_review['Polarity'].apply(mapScore)

In [154]:
data_review['TextBlobAnalysis'].value_counts()

Positive    15236
Negative     2511
Neutral      1413
Name: TextBlobAnalysis, dtype: int64

### Sentiment Analysis using VADER
VADER stands for Valence Aware Dictionary and Sentiment Reasoner. Vader sentiment not only tells if the statement is positive or negative along with the intensity of emotion.

It uses a list of lexical features (e.g. word) which are labeled as positive or negative according to their semantic orientation to calculate the text sentiment.   

Vader sentiment returns the probability of a given input sentence to be 

Positive, negative, and neutral. 

Vader is optimized for social media data and can yield good results when used with data from twitter, facebook, etc.

The main drawback with the rule-based approach for sentiment analysis is that the method only cares about individual words and completely ignores the context in which it is used. 

For example, “the party was savage” will be negative when considered by any token-based algorithms.

In [155]:
# function to calculate vader sentiment
def vadersentimentanalysis(review):
    vs = analyzer.polarity_scores(review)
    return vs['compound']
    
# function to analyse
def vader_analysis(compound):
    if compound >= 0.5:
        return 'Positive'
    elif compound <= -0.5 :
        return 'Negative'
    else:
        return 'Neutral'


In [156]:
analyzer = SentimentIntensityAnalyzer()
data_review['Vader Sentiment'] = data_review['Lemma'].apply(vadersentimentanalysis)
data_review['Vader Analysis'] = data_review['Vader Sentiment'].apply(vader_analysis)
data_review['Vader Analysis'].value_counts()

Positive    9898
Neutral     7562
Negative    1700
Name: Vader Analysis, dtype: int64

In [157]:
data_review

,review,review_clean,POS tagged,Lemma,Subjectivity,Polarity,TextBlobAnalysis,Vader Sentiment,Vader Analysis
0,value for money if you are looking for a budge...,value for money if you are looking for a budge...,"[(value, n), (for, None), (money, n), (if, Non...",value for money if you be look for a budget ...,0.640476,-0.162202,Negative,0.5182,Positive
1,works a like a charm in this price range i get...,works a like a charm in this price range i get...,"[(works, v), (a, None), (like, None), (a, None...",work a like a charm in this price range i ge...,0.366667,0.183333,Positive,0.7964,Positive
2,value for money i bought this for my mother wh...,value for money i bought this for my mother wh...,"[(value, n), (for, None), (money, n), (i, n), ...",value for money i buy this for my mother who...,0.494569,0.294864,Positive,0.7343,Positive
3,good budget mobile good enough mobile in this ...,good budget mobile good enough mobile in this ...,"[(good, a), (budget, n), (mobile, n), (good, a...",good budget mobile good enough mobile in thi...,0.566667,0.466667,Positive,0.7003,Positive
4,camera only camera quality not good,camera only camera quality not good,"[(camera, n), (only, r), (camera, n), (quality...",camera only camera quality not good,0.800000,-0.175000,Negative,-0.3412,Neutral
...,...,...,...,...,...,...,...,...,...
19946,good phone superb quality mobile phone and cam...,good phone superb quality mobile phone and cam...,"[(good, a), (phone, n), (superb, n), (quality,...",good phone superb quality mobile phone and c...,0.633333,0.900000,Positive,0.9042,Positive
19947,exslent mobile sarvis is good,exslent mobile sarvis is good,"[(exslent, n), (mobile, n), (sarvis, n), (is, ...",exslent mobile sarvis be good,0.600000,0.700000,Positive,0.4404,Neutral
19948,best product in this price range good battery ...,best product in this price range good battery ...,"[(best, a), (product, n), (in, None), (this, N...",best product in this price range good batter...,0.500000,0.800000,Positive,-0.1262,Neutral
19949,very fast delivery awesome,very fast delivery awesome,"[(very, r), (fast, r), (delivery, n), (awesome...",very fast delivery awesome,0.890000,0.630000,Positive,0.6557,Positive


In [158]:
def sentiwordnetanalysis(pos_data):
    sentiment = 0
    tokens_count = 0
    for word, pos in pos_data:
        if not pos:
            continue
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        if not lemma:
            continue
        
        synsets = wordnet.synsets(lemma, pos=pos)
        if not synsets:
            continue

        # Take the first sense, the most common
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        sentiment += swn_synset.pos_score() - swn_synset.neg_score()
        tokens_count += 1
        # print(swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score())
    if not tokens_count:
        return 0
    if sentiment>0:
        return "Positive"
    if sentiment==0:
        return "Neutral"
    else:
        return "Negative"

In [159]:
data_review['SWN analysis'] = data_review['POS tagged'].apply(sentiwordnetanalysis)
data_review['SWN analysis'].value_counts()

Positive    14426
Negative     3134
Neutral      1438
0             162
Name: SWN analysis, dtype: int64

In [160]:
data_review[data_review['SWN analysis']==0]

,review,review_clean,POS tagged,Lemma,Subjectivity,Polarity,TextBlobAnalysis,Vader Sentiment,Vader Analysis,SWN analysis
101,taimpass,taimpass,"[(taimpass, n)]",taimpass,0.00,0.0,Neutral,0.0000,Neutral,0
279,everything like,everything like,"[(everything, n), (like, None)]",everything like,0.00,0.0,Neutral,0.3612,Neutral,0
371,excellent,excellent,"[(excellent, n)]",excellent,1.00,1.0,Positive,0.5719,Positive,0
409,exloent supar,exloent supar,"[(exloent, n), (supar, n)]",exloent supar,0.00,0.0,Neutral,0.0000,Neutral,0
419,sparsh,sparsh,"[(sparsh, n)]",sparsh,0.00,0.0,Neutral,0.0000,Neutral,0
...,...,...,...,...,...,...,...,...,...,...
19202,excellent,excellent,"[(excellent, n)]",excellent,1.00,1.0,Positive,0.5719,Positive,0
19204,awesome,awesome,"[(awesome, n)]",awesome,1.00,1.0,Positive,0.6249,Positive,0
19357,excellent,excellent,"[(excellent, n)]",excellent,1.00,1.0,Positive,0.5719,Positive,0
19615,exellent,exellent,"[(exellent, n)]",exellent,0.00,0.0,Neutral,0.0000,Neutral,0


### Topic Modelling

In [19]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [14]:
data_tm = data[['review']]

In [15]:
data = data_tm.review.values.tolist()  

In [21]:
data[:1]

['value for money if you are looking for a budget phone within k go for this one this one delivers the promises they make and stands upto the expectation only the battery performance is little disappointing mah with no fast charging option but that does not make this one a showstopper unless you expect a beast out of below k budget smartphone with gb gb config with octa core non snapdragon processor this phone is the smart est choice for you at least at this moment in the market']

In [22]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['value', 'for', 'money', 'if', 'you', 'are', 'looking', 'for', 'budget', 'phone', 'within', 'go', 'for', 'this', 'one', 'this', 'one', 'delivers', 'the', 'promises', 'they', 'make', 'and', 'stands', 'upto', 'the', 'expectation', 'only', 'the', 'battery', 'performance', 'is', 'little', 'disappointing', 'mah', 'with', 'no', 'fast', 'charging', 'option', 'but', 'that', 'does', 'not', 'make', 'this', 'one', 'showstopper', 'unless', 'you', 'expect', 'beast', 'out', 'of', 'below', 'budget', 'smartphone', 'with', 'gb', 'gb', 'config', 'with', 'octa', 'core', 'non', 'snapdragon', 'processor', 'this', 'phone', 'is', 'the', 'smart', 'est', 'choice', 'for', 'you', 'at', 'least', 'at', 'this', 'moment', 'in', 'the', 'market']]


In [25]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data_words = remove_stopwords(data_words)
print(data_words[:1])

[['value', 'money', 'looking', 'budget', 'phone', 'within', 'go', 'one', 'one', 'delivers', 'promises', 'make', 'stands', 'upto', 'expectation', 'battery', 'performance', 'little', 'disappointing', 'mah', 'fast', 'charging', 'option', 'make', 'one', 'showstopper', 'unless', 'expect', 'beast', 'budget', 'smartphone', 'gb', 'gb', 'config', 'octa', 'core', 'non', 'snapdragon', 'processor', 'phone', 'smart', 'est', 'choice', 'least', 'moment', 'market']]


In [26]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['value', 'money', 'looking', 'budget', 'phone', 'within', 'go', 'one', 'one', 'delivers', 'promises', 'make', 'stands', 'upto', 'expectation', 'battery', 'performance', 'little', 'disappointing', 'mah', 'fast', 'charging', 'option', 'make', 'one', 'showstopper', 'unless', 'expect', 'beast', 'budget', 'smartphone', 'gb', 'gb', 'config', 'octa_core', 'non', 'snapdragon', 'processor', 'phone', 'smart', 'est', 'choice', 'least', 'moment', 'market']


In [27]:
# Define function for stopwords, bigrams, trigrams and lemmatization

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [28]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['value', 'money', 'look', 'budget', 'phone', 'go', 'deliver', 'promise', 'make', 'stand', 'upto', 'expectation', 'battery', 'performance', 'little', 'disappointing', 'fast', 'charge', 'option', 'make', 'showstopper', 'expect', 'beast', 'budget', 'config', 'non', 'snapdragon', 'processor', 'phone', 'smart', 'e', 'choice', 'least', 'moment', 'market']]


In [29]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)]]


In [30]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('battery', 1),
  ('beast', 1),
  ('budget', 2),
  ('charge', 1),
  ('choice', 1),
  ('config', 1),
  ('deliver', 1),
  ('disappointing', 1),
  ('e', 1),
  ('expect', 1),
  ('expectation', 1),
  ('fast', 1),
  ('go', 1),
  ('least', 1),
  ('little', 1),
  ('look', 1),
  ('make', 2),
  ('market', 1),
  ('moment', 1),
  ('money', 1),
  ('non', 1),
  ('option', 1),
  ('performance', 1),
  ('phone', 2),
  ('processor', 1),
  ('promise', 1),
  ('showstopper', 1),
  ('smart', 1),
  ('snapdragon', 1),
  ('stand', 1),
  ('upto', 1),
  ('value', 1)]]

In [31]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [33]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.125*"reader" + 0.120*"finger_print" + 0.114*"back" + 0.112*"receive" + '
  '0.094*"cover" + 0.080*"build" + 0.045*"rs" + 0.023*"offer" + '
  '0.014*"goodcamera" + 0.014*"bright"'),
 (1,
  '0.607*"mobile" + 0.174*"awesome" + 0.128*"worth" + 0.007*"gift" + '
  '0.004*"father" + 0.000*"amazon" + 0.000*"superb" + 0.000*"day" + '
  '0.000*"display" + 0.000*"amazing"'),
 (2,
  '0.333*"buy" + 0.141*"slow" + 0.117*"thank" + 0.069*"normal" + 0.059*"app" + '
  '0.052*"user" + 0.029*"start" + 0.024*"recommend" + 0.015*"sometimes" + '
  '0.013*"youtube"'),
 (3,
  '0.472*"fingerprint" + 0.107*"available" + 0.000*"sensor" + 0.000*"amazon" + '
  '0.000*"cool" + 0.000*"simply" + 0.000*"amazing" + 0.000*"future" + '
  '0.000*"lock" + 0.000*"day"'),
 (4,
  '0.335*"quality" + 0.323*"camera" + 0.173*"bad" + 0.056*"look" + '
  '0.052*"backup" + 0.014*"hai" + 0.000*"amazon" + 0.000*"display" + '
  '0.000*"superb" + 0.000*"day"'),
 (5,
  '0.196*"poor" + 0.154*"much" + 0.104*"smartphone" + 0.053*"pe

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.195543566449219
